# Project 8 -- Ritvik Indupuri

**TA Help:** John Smith, Alice Jones

- Help with figuring out how to write a function.
    
**Collaboration:** Friend1, Friend2
    
- Helped figuring out how to load the dataset.
- Helped debug error with my plot.

## Question 1

In [1]:
import sqlite3

# I'm connecting to the read-only source database.
read_only_lahman = sqlite3.connect('/anvil/projects/tdm/data/lahman/lahman.db')

# Now, I'm creating a new, empty database file in my current directory.
local_lahman = sqlite3.connect('./lahman.db')

# The backup() function copies everything from the source to my new local file.
read_only_lahman.backup(local_lahman)

# It's good practice to close the connections when I'm done.
read_only_lahman.close()
local_lahman.close()

In [3]:
%%sql sqlite:///./lahman.db
-- Selecting all columns (*) from the Teams table, but only the first 5 rows.
SELECT * FROM Teams LIMIT 5;

Done.


yearID,lgID,teamID,franchID,divID,teamRank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
1871,NA,BS1,BNA,,3,31,,20,10,,,N,,401,1372,426,70,37,3,60,19,73,16,,,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,Boston Red Stockings,South End Grounds I,,103,98,BOS,BS1,BS1
1871,NA,CH1,CNA,,2,28,,19,9,,,N,,302,1196,323,52,21,10,60,22,69,21,,,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,,104,102,CHI,CH1,CH1
1871,NA,CL1,CFC,,8,29,,10,19,,,N,,249,1186,328,35,40,7,26,25,18,8,,,341,116,4.11,23,0,0,762,346,13,53,34,234,15,0.818,Cleveland Forest Citys,National Association Grounds,,96,100,CLE,CL1,CL1
1871,NA,FW1,KEK,,7,19,,7,12,,,N,,137,746,178,19,8,2,33,9,16,4,,,243,97,5.17,19,1,0,507,261,5,21,17,163,8,0.803,Fort Wayne Kekiongas,Hamilton Field,,101,107,KEK,FW1,FW1
1871,NA,NY2,NNA,,5,33,,16,17,,,N,,302,1404,403,43,21,1,33,15,46,15,,,313,121,3.72,32,1,0,879,373,7,42,22,235,14,0.84,New York Mutuals,Union Grounds (Brooklyn),,90,88,NYU,NY2,NY2


My first step was to create a personal, writable copy of the Lahman database using a Python script. This is a crucial starting point because the original database file is read-only, and this project requires me to modify it by creating and deleting tables. After creating the local lahman.db file, I connected my SQL session to it using the connection string in the magic command. Running a simple SELECT query on the Teams table served as a quick, successful test to confirm that the database was copied correctly and that my connection is working.



## Question 2

In [6]:
%%sql
-- This first line prevents an error if the table already exists.
DROP TABLE IF EXISTS MyPlayers;

-- This creates the new table with its column definitions.
CREATE TABLE MyPlayers (
    playerID TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    teamID TEXT
);

 * sqlite:///./lahman.db
Done.
Done.


[]

In [7]:
%%sql
INSERT INTO MyPlayers (playerID, name, teamID)
VALUES ('johndo01', 'John Doe', 'NYA');

 * sqlite:///./lahman.db
1 rows affected.


[]

In [8]:
%%sql
INSERT INTO MyPlayers (playerID, name, teamID)
VALUES
    ('janesm01', 'Jane Smith', 'BOS'),
    ('bobjoh01', 'Bob Johnson', 'CHA'),
    ('alicbr01', 'Alice Brown', 'LAN');

 * sqlite:///./lahman.db
3 rows affected.


[]

In [13]:
%%sql
-- Using INSERT OR IGNORE to skip duplicates I added manually.
INSERT OR IGNORE INTO MyPlayers (playerID, name, teamID)
SELECT
    p.playerID,
    p.nameFirst || ' ' || p.nameLast AS name,
    MIN(s.teamID) as teamID
FROM People p
JOIN Salaries s on s.playerID = p.playerID
WHERE s.yearID = ( SELECT MAX(s2.yearID) FROM Salaries s2 WHERE s2.playerID = s.playerID )
GROUP BY p.playerID;

 * sqlite:///./lahman.db
0 rows affected.


[]

In [15]:
%%sql
SELECT * FROM MyPlayers LIMIT 5;

 * sqlite:///./lahman.db
Done.


playerID,name,teamID
johndo01,John Doe,NYA
janesm01,Jane Smith,BOS
bobjoh01,Bob Johnson,CHA
alicbr01,Alice Brown,LAN
aardsda01,David Aardsma,NYA


In this question, I learned the fundamentals of creating a new table and populating it. The CREATE TABLE statement is where I defined the table's "schema"—its columns and data types. Making playerID a PRIMARY KEY is an important data integrity rule that ensures no two players can have the same ID. I then practiced adding data using INSERT INTO, first one row at a time and then multiple rows, which is more efficient. The most powerful technique was using INSERT INTO ... SELECT, which allowed me to populate my new table with thousands of rows of processed data from existing tables. This is a very common workflow for creating summary or custom tables.

## Question 3

In [17]:
%%sql
ALTER TABLE MyPlayers ADD COLUMN height REAL;

 * sqlite:///./lahman.db
(sqlite3.OperationalError) duplicate column name: height
[SQL: ALTER TABLE MyPlayers ADD COLUMN height REAL;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [18]:
%%sql
ALTER TABLE MyPlayers ADD COLUMN weight REAL;

 * sqlite:///./lahman.db
Done.


[]

In [19]:
%%sql
ALTER TABLE MyPlayers ADD COLUMN age INTEGER;

 * sqlite:///./lahman.db
Done.


[]

In [21]:
%%sql
INSERT INTO MyPlayers (playerID, name, teamID, height, weight, age)
VALUES
    ('mikejo01', 'Mike Jordan', 'ATL', 78.5, 220.0, 32),
    ('daviro01', 'David Robinson', 'SFN', 85.0, 235.0, 28),
    ('scotpi01', 'Scottie Pippen', 'CHN', 80.0, 210.5, 29),
    ('karlma01', 'Karl Malone', 'UTA', 81.0, 250.0, 31),
    ('patrick01', 'Patrick Ewing', 'NYN', 84.0, 240.0, 30);

 * sqlite:///./lahman.db
(sqlite3.IntegrityError) UNIQUE constraint failed: MyPlayers.playerID
[SQL: INSERT INTO MyPlayers (playerID, name, teamID, height, weight, age)
VALUES
    ('mikejo01', 'Mike Jordan', 'ATL', 78.5, 220.0, 32),
    ('daviro01', 'David Robinson', 'SFN', 85.0, 235.0, 28),
    ('scotpi01', 'Scottie Pippen', 'CHN', 80.0, 210.5, 29),
    ('karlma01', 'Karl Malone', 'UTA', 81.0, 250.0, 31),
    ('patrick01', 'Patrick Ewing', 'NYN', 84.0, 240.0, 30);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [22]:
%%sql
SELECT *
FROM MyPlayers
ORDER BY height DESC
LIMIT 5;

 * sqlite:///./lahman.db
Done.


playerID,name,teamID,height,weight,age
daviro01,David Robinson,SFN,85.0,235.0,28
patrick01,Patrick Ewing,NYN,84.0,240.0,30
karlma01,Karl Malone,UTA,81.0,250.0,31
scotpi01,Scottie Pippen,CHN,80.0,210.5,29
mikejo01,Mike Jordan,ATL,78.5,220.0,32


This question focused on modifying a table's structure after it's been created. The ALTER TABLE command is used for this, and I used it to add new columns (height, weight, and age) to my existing MyPlayers table. This is a realistic scenario, as data requirements often change over time. After altering the schema, I was able to insert new rows that contained values for these new columns. Finally, I queried this new data, using ORDER BY height DESC to create a simple leaderboard of the tallest players I had added.

## Question 4

In [23]:
%%sql
DELETE FROM MyPlayers
WHERE playerID = 'johndo01';

 * sqlite:///./lahman.db
1 rows affected.


[]

In [24]:
%%sql
DELETE FROM MyPlayers
WHERE height < 72.0;

 * sqlite:///./lahman.db
0 rows affected.


[]

In [25]:
%%sql
-- This should return no rows, confirming the player is gone.
SELECT * FROM MyPlayers WHERE playerID = 'johndo01';

 * sqlite:///./lahman.db
Done.


playerID,name,teamID,height,weight,age


In [26]:
%%sql
-- This should also return no rows, confirming the height-based deletion worked.
SELECT * FROM MyPlayers WHERE height < 72.0;

 * sqlite:///./lahman.db
Done.


playerID,name,teamID,height,weight,age


In this quetsion, I practiced removing specific rows from a table. The key takeaway is the importance of the DELETE command's WHERE clause. Without a WHERE clause, DELETE removes all rows, so it's crucial to be specific about what you want to remove. I first deleted a single row by its unique playerID and then deleted a group of rows that matched a specific condition (height < 72.0). Verifying the deletion by trying to SELECT the removed data is a critical final step to ensure the command worked as intended.

## Question 5

In [27]:
%%sql
-- Using DELETE without a WHERE clause to empty the table.
DELETE FROM MyPlayers;

 * sqlite:///./lahman.db
5157 rows affected.


[]

In [28]:
%%sql
DROP TABLE MyPlayers;

 * sqlite:///./lahman.db
Done.


[]

In [29]:
%%sql
SELECT * FROM MyPlayers LIMIT 5;

 * sqlite:///./lahman.db
(sqlite3.OperationalError) no such table: MyPlayers
[SQL: SELECT * FROM MyPlayers LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


The final question covered the most destructive SQL operations. I first used DELETE without a WHERE clause, which empties a table by removing all its rows but leaves the table's structure intact for future use. Then, I used DROP TABLE, which is far more permanent—it removes the entire table and all its data from the database. The best way to confirm that DROP TABLE worked is to try to query the table again. The resulting "no such table" error is the expected outcome and serves as definitive proof that the table is gone. 

## Pledge

By submitting this work I hereby pledge that this is my own, personal work. I've acknowledged in the designated place at the top of this file all sources that I used to complete said work, including but not limited to: online resources, books, and electronic communications. I've noted all collaboration with fellow students and/or TA's. I did not copy or plagiarize another's work.

> As a Boilermaker pursuing academic excellence, I pledge to be honest and true in all that I do. Accountable together – We are Purdue.